# Création de prompts pour le nommage et la description des clusters

In [1]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_composition.rds")

[1] "fr_CA.UTF-8/fr_CA.UTF-8/fr_CA.UTF-8/C/fr_CA.UTF-8/C"


Attachement du package : ‘dplyr’


Les objets suivants sont masqués depuis ‘package:stats’:

    filter, lag


Les objets suivants sont masqués depuis ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_femmes     <- mean(cluster_data$female, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$langEn, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$langFr, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethn_White, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_hetero, na.rm = TRUE) * 100
  educ_mean       <- mean(cluster_data$educ, na.rm = TRUE)
  age_mean        <- mean(cluster_data$age, na.rm = TRUE) * 80
  revenu_moyen    <- mean(cluster_data$ses_income, na.rm = TRUE) * 225000

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
    "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona ", cluster_value, " :\n",

    "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
    if (any(cluster_vars$color == "green")) {
      paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
    } else {
      "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
    },
    "\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
    if (any(cluster_vars$color == "red")) {
      paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
    } else {
      "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
    },
    "\n\nCe cluster est composé de \n", 
    "Femmes: ", round(perc_femmes, 1), "%, 
     Immigrants: ", round(perc_immigrants, 1), "%\n",
    "Âge: ", round(age_mean, 1), " ans, 
     Revenu: ", round(revenu_moyen, 1), " k$\n",
    "Langues: 
     En ", round(perc_langEn, 1), "%, 
     Fr ", round(perc_langFr, 1), "%, 
     Autres: ", round(perc_ses_languageOther, 1), "%\n",
    "Édu: ", round(educ_mean, 1), " (0=bas, 1=élevé), 
     Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
    "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n"
  ))

  # Stocker le prompt dans la liste
  prompts[[cluster_value]] <- prompt_description
}

Warning message in mean.default(cluster_data$female, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$immigrant, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$langEn, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$langFr, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$ses_ethn_White, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$ses_hetero, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$educ, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$age, na.rm = TRUE):
“l'argument n'est ni numérique, n

In [3]:
prompts

$`1`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 1 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"ses_age\", \"ses_languageEnglish\")\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nAucune variable avec une moyenne significativement plus basse que la moyenne de la population en général.\n\nCe cluster est composé de \nFemmes: NA%, \n     Immigrants: NA%\nÂge: NA ans, \n     Revenu: 122569.4 k$\nLangues: \n     En NA%, \n     Fr NA%, \n     Autres: 0%\nÉdu: NA (0=bas, 1=élevé), \n     Ethnie (Blanc): NA%\nOrientation (Hétéro): NA%\n"

$`10`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 10 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_exerciseGym\", \"lifestyle_goFishingFreq_numeric\", \"lifestyle_goHuntingFreq_numeric\", \"lifestyle_goMuseumsFreq_numeric\", \"lifestyle_motorizedActFreq_numeric\", \"lifestyle_smokeFreq\", \"lifestyle_volunteeringFreq_numeric\")\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_clothingStyleCasual\", \"lifestyle_exerciseWalk\", \"ses_age\")\n\nCe cluster est composé de \nFemmes: NA%, \n     Immigrants: NA%\nÂge: NA ans, \n     Revenu: 124522.9 k$\nLangues: \n     En NA%, \n     Fr NA%, \n     Autres: 0%\nÉdu: NA (0=bas, 1=élevé), \n     Ethnie (Blanc): NA%\nOrientation (Hétéro): NA%\n"

$`2`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 2 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nlifestyle_typeTransportActive\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_goFishingFreq_numeric\", \"lifestyle_typeTransportCar\", \"ses_income\")\n\nCe cluster est composé de \nFemmes: NA%, \n     Immigrants: NA%\nÂge: NA ans, \n     Revenu: 95122 k$\nLangues: \n     En NA%, \n     Fr NA%, \n     Autres: 0%\nÉdu: NA (0=bas, 1=élevé), \n     Ethnie (Blanc): NA%\nOrientation (Hétéro): NA%\n"

$`3`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 3 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nses_sexOrientationQueer\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"ses_age\", \"ses_sexOrientationHetero\")\n\nCe cluster est composé de \nFemmes: NA%, \n     Immigrants: NA%\nÂge: NA ans, \n     Revenu: 101470.6 k$\nLangues: \n     En NA%, \n     Fr NA%, \n     Autres: 0%\nÉdu: NA (0=bas, 1=élevé), \n     Ethnie (Blanc): NA%\nOrientation (Hétéro): NA%\n"

$`4`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 4 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_clothingStyleOther\", \"lifestyle_hasTattoos\")\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nlifestyle_clothingStyleCasual\n\nCe cluster est composé de \nFemmes: NA%, \n     Immigrants: NA%\nÂge: NA ans, \n     Revenu: 98437.5 k$\nLangues: \n     En NA%, \n     Fr NA%, \n     Autres: 2.5%\nÉdu: NA (0=bas, 1=élevé), \n     Ethnie (Blanc): NA%\nOrientation (Hétéro): NA%\n"

$`5`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 5 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_exerciseYoga\", \"ses_income\")\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_

In [4]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [5]:
reponses_openai

$`1`
[1] "Pour une personne habitant au Québec correspondant au persona 1, nous pouvons proposer le prénom \"Emily.\" Voici une brève description pour ce persona :\n\nEmily est une femme qui réside au Québec avec un âge supérieur à la moyenne générale de la population. Elle parle principalement anglais, ce qui est également plus fréquent que dans la population générale. Elle bénéficie d'un revenu élevé, avec un revenu moyen de 122,569.4 dollars, bien au-dessus de la moyenne. Bien que nous n'ayons pas d'autres données spécifiques, on peut supposer qu'elle est bien intégrée et probablement engagée dans une carrière professionnelle qui correspond à son niveau de revenu. Emily pourrait travailler dans un environnement où l'anglais est prédominant, peut-être dans une entreprise internationale ou dans un secteur où l'interaction avec des clients anglophones est courante. Elle est probablement bien éduquée, car le revenu élevé est souvent corrélé avec un niveau d'éducation élevé."

$`10`
[1] "Pour une personne habitant au Québec correspondant au persona 10, je propose le prénom suivant : **Laurence**. Ce prénom est à la fois classique et moderne, et il est courant au Québec, ce qui le rend bien adapté à une femme de cette région.\n\n### Brève Description pour Laurence\n\n**Lifestyle et Activités**\n- **Sportive et Active** : Laurence fréquente régulièrement la salle de gym et participe à des activités motorisées. Elle aime garder la forme et se lance souvent dans des aventures en plein air comme la pêche et la chasse.\n- **Intérêt Culturel** : Elle visite fréquemment des musées, ce qui montre son intérêt pour l'art et la culture.\n- **Engagée dans la communauté** : Laurence est impliquée dans des activités de bénévolat, contribuant ainsi à sa communauté.\n\n**Habitudes**\n- **Fumeuse** : Elle fume plus que la moyenne de la population.\n- **Style Vestimentaire** : Moins axée sur le style décontracté, elle préfère peut-être des tenues plus sophistiquées ou adaptées à ses diverses activités.\n\n**Démographie et Socio-économie**\n- **Revenu Élevé** : Avec un revenu de 124 522,9 k$, Laurence vit confortablement et a la liberté de s'engager dans diverses activités.\n- **Jeune Adulte** : Dans la tranche d'âge plus jeune que la moyenne, Laurence est dynamique et pleine d'énergie pour mener ses multiples activités.\n\n**Langues et Éducation**\n- Bien qu'il n'y ait pas d'information précise sur les langues, elle est probablement francophone ou bilingue, vu le contexte québécois.\n- **Éducation Élevée** : Laurence possède un niveau d'éducation supérieur, ce qui l'aide à naviguer et exceller dans divers aspects de sa vie.\n\nLaurence est une femme moderne, active et engagée, avec un mode de vie bien rempli, typique de certaines jeunes professionnelles québécoises."

$`2`
[1] "Pour une personne habitant au Québec, un prénom qui pourrait convenir est \"Élise\". \n\n**Persona 2 : Élise**\n\n- **Transport et style de vie :** Élise privilégie les transports actifs. Elle préfère marcher, faire du vélo ou utiliser des transports en commun pour ses déplacements quotidiens. Elle a peu d'intérêt pour la pêche et n'utilise guère la voiture, ce qui réduit son empreinte carbone et montre son engagement envers un style de vie plus durable.\n\n- **Revenu :** Bien que son revenu soit significativement plus bas que la moyenne de la population en général, avec 95,122 k$, Élise parvient à mener une vie confortable en choisissant des activités et des modes de transport qui ne nécessitent pas de grosses dépenses.\n\n- **Langues :** Élise parle principalement le français, avec une compréhension de l'anglais, et elle ne parle pas d'autres langues.\n\n- **Éducation et travail :** Bien que le niveau d'éducation ne soit pas spécifié, on peut imaginer qu'Élise a un niveau d'éducation modéré à élevé, qui lui permet de vivre confortablement avec son revenu actuel.\n\n- **Mode de vie et valeurs :** Élise est probablement tournée vers des valeurs écologiques et durables. Son 

In [6]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [7]:
description_clusters

,prompt,description
,<chr>,<chr>
1,"Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 1 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""ses_age"", ""ses_languageEnglish"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général. Ce cluster est composé de Femmes: NA%, Immigrants: NA% Âge: NA ans, Revenu: 116452 k$ Langues: En NA%, Fr NA%, Autres: 0% Édu: NA (0=bas, 1=élevé), Ethnie (Blanc): NA% Orientation (Hétéro): NA%","Pour une personne habitant au Québec avec les caractéristiques mentionnées, je propose le prénom ""Charlotte"". **Persona 1 : Charlotte** - **Âge et Revenu:** Charlotte appartient à un groupe avec un âge moyen plus élevé que la population générale et a un revenu annuel significativement élevé de 116 452 $. - **Langue:** Charlotte parle majoritairement l'anglais, ce qui est également une caractéristique notable de ce groupe. - **Culture et Identité:** Bien que des données comme le pourcentage de femmes, d'immigrants, l'éducation, et l'ethnicité ne soient pas précisées, Charlotte peut être perçue comme une personne ayant probablement un niveau d'éducation élevé vu son revenu, et qui valorise la diversité linguistique grâce à sa maîtrise de l'anglais. - **Vie au Québec:** Habitant au Québec, Charlotte pourrait être impliquée dans des activités qui favorisent l'intégration linguistique et culturelle, participant à des événements où l'anglais est prédominant tout en appréciant l'environnement francophone. - **Profession:** Avec son revenu et ses compétences linguistiques, Charlotte pourrait travailler dans un domaine comme les affaires, la technologie, ou les communications internationales, où l'anglais est souvent utilisé. - **Mode de Vie:** Elle mène une vie confortable, avec des intérêts potentiels pour le voyage, la culture, et des activités sociales qui lui permettent de capitaliser sur ses compétences linguistiques et son réseau professionnel. Avec ces éléments, Charlotte est un persona qui représente une personne bien intégrée au Québec, tout en apportant une perspective anglophone à sa communauté."
2,"Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 2 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""ses_age"", ""ses_languageFrench"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""lifestyle_goHuntingFreq_numeric"", ""lifestyle_volunteeringFreq_numeric"", ""ses_languageEnglish"") Ce cluster est composé de Femmes: NA%, Immigrants: NA% Âge: NA ans, Revenu: 104135.7 k$ Langues: En NA%, Fr NA%, Autres: 0.4% Édu: NA (0=bas, 1=élevé), Ethnie (Blanc): NA% Orientation (Hétéro): NA%","Pour une personne habitant au Québec, un prénom qui résonne bien est ""Élise"". **Persona 2 : Élise** - **Âge :** Élise est significativement plus âgée que la moyenne de la population. - **Langue :** Élise parle majoritairement le français, ce qui est beaucoup plus élevé par rapport à la moyenne de la population. Elle parle peu l'anglais. - **Revenu :** Elle gagne un revenu confortable de 104 135,7 k$ par an. - **Style de vie :** Élise participe rarement à des activités de chasse et ne s'engage pas fréquemment dans le bénévolat. - **Diversité linguistique :** Bien qu’elle parle majoritairement français, une infime partie (0,4%) pourrait parler une autre langue que le français et l'anglais. - **Origine et culture :** Les données concernant son statut d'immigrant, son niveau d'éducation, et son appartenance ethnique ne sont pas spécifiées. Élise est donc une femme francophone, relativement plus âgée, avec un revenu élevé, vivant probablement en milieu urbain ou suburbain. Elle est plutôt privée et préfère des activi

In [8]:
write.csv(description_clusters, "description_clusters.csv", row.names = FALSE)

In [9]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_description.rds")